In [1]:
import numpy as np

# Activation functions
def relu(x):
    """ReLU activation: max(0, x) applied elementwise."""
    return np.maximum(0, x)

def tanh(x):
    """tanh activation: (e^x - e^{-x})/(e^x + e^{-x}) applied elementwise."""
    return np.tanh(x)

def softmax(x):
    """Softmax activation for output layer (assumes x is 1D or 2D batch of scores)."""
    # For numeric stability, subtract max
    x_shifted = x - np.max(x, axis=-1, keepdims=True)
    exp_x = np.exp(x_shifted)
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

In [ ]:
def conv_forward(X, W, b):
    """
    Convolutional layer forward pass.
    X: input array of shape (C_in, H_in, W_in)
    W: weight/filter array of shape (C_out, C_in, kH, kW)
    b: bias array of shape (C_out,)
    Returns: output feature map of shape (C_out, H_out, W_out)

    TODO: use numpy for simplifying for loops
    """
    C_out, C_in, kH, kW = W.shape
    _, H_in, W_in = X.shape
    H_out = H_in - kH + 1  # using stride 1, no padding
    W_out = W_in - kW + 1
    # Initialize output volume
    out = np.zeros((C_out, H_out, W_out))
    # Convolution: slide each filter over the input
    for oc in range(C_out):             # for each output channel (filter)
        for i in range(H_out):          # slide vertically
            for j in range(W_out):      # slide horizontally
                # current region of input of shape (C_in, kH, kW)
                region = X[:, i:i+kH, j:j+kW]
                # element-wise multiply and sum -> dot product
                out[oc, i, j] = np.sum(region * W[oc]) + b[oc]
    return out

# Example usage:
# X has shape (1, 32, 32) for a single grayscale image, W shape (6, 1, 5, 5), b shape (6,)
X = np.random.rand(1, 32, 32)  # dummy input
W1 = np.random.rand(6, 1, 5, 5)
b1 = np.random.rand(6,)
out1 = conv_forward(X, W1, b1)
print(out1)  # should be (6, 28, 28)

out1_act = tanh(out1)


[[[7.74466202 7.87499121 6.46599859 ... 7.16033799 7.23879439 7.57612002]
  [6.7638502  7.60400893 6.76476533 ... 7.40983525 7.83721912 7.51518527]
  [7.7023605  7.90636587 8.46711908 ... 7.1873372  5.81481626 6.23583491]
  ...
  [6.2697161  6.11070863 7.97203259 ... 5.69440666 5.42024257 4.96823978]
  [6.58880314 5.96099481 7.55421972 ... 4.98563466 5.21394182 4.52339772]
  [6.48581985 5.82830623 8.03175372 ... 6.29752652 5.23555688 6.29593642]]

 [[5.93311404 7.44329115 6.54410268 ... 6.66468612 6.09950863 6.35260915]
  [6.96811135 6.81049245 6.22031258 ... 6.19732251 6.06148532 6.96708839]
  [6.88897563 6.53026876 6.79292122 ... 5.9612273  6.28533861 6.0379001 ]
  ...
  [6.57106862 5.69265337 6.36471862 ... 4.44734034 5.33227731 4.17142554]
  [5.92534058 4.65587737 6.39839314 ... 4.34143261 4.75927957 4.72974404]
  [6.60682917 5.59506476 5.83736486 ... 4.37604685 5.76264114 5.0160156 ]]

 [[8.09944941 8.61101523 7.09485369 ... 7.38722896 7.70363995 8.54063998]
  [7.30421841 6.783711

In [11]:
def avg_pool_forward(X):
    """
    Average pooling forward pass (2x2 pool, stride 2).
    X: input array of shape (C, H, W)
    Returns: output array of shape (C, H/2, W/2)
    """
    C, H, W = X.shape
    # Assuming H and W are even and divisible by 2 for simplicity
    out = np.zeros((C, H//2, W//2))
    for c in range(C):
        for i in range(0, H, 2):         # step by 2
            for j in range(0, W, 2):     # step by 2
                patch = X[c, i:i+2, j:j+2]        # 2x2 region
                out[c, i//2, j//2] = np.mean(patch)
    return out

# Example usage:
X_pool_in = np.random.rand(6, 28, 28)  # e.g. output from conv layer (6,28,28)
out_pool = avg_pool_forward(X_pool_in)
print(out_pool.shape)  # should be (6, 14, 14)

(6, 14, 14)


In [13]:
def fc_forward(x, W, b):
    """
    Fully-connected layer forward pass.
    x: input vector of shape (N_in,)
    W: weight matrix of shape (N_out, N_in)
    b: bias vector of shape (N_out,)
    Returns: output vector of shape (N_out,)
    """
    return W.dot(x) + b

# Example usage:
x = np.random.rand(120,)    # input from previous layer (flattened conv output)
W4 = np.random.rand(84, 120)
b4 = np.random.rand(84,)
out_fc = fc_forward(x, W4, b4)
print(out_fc.shape)  # should be (84,)


(84,)


In [ ]:
def lenet5_forward(image):
    """
    Forward pass for the entire LeNet-5 model on a single image.
    image: input image array of shape (1, 32, 32)  - 1 channel, 32x32 (assuming already zero-padded to 32x32 if originally 28x28).
    Returns: output probabilities for 10 classes (softmax output vector of shape (10,))
    """
    # Layer C1: Conv 5x5 -> 6 feature maps, then activation
    out_c1 = conv_forward(image, W1, b1)         # shape (6, 28, 28)
    out_c1 = tanh(out_c1)                        # apply tanh or relu
    
    # Layer S2: 2x2 Pooling -> 6 feature maps
    out_s2 = avg_pool_forward(out_c1)            # shape (6, 14, 14)
    
    # Layer C3: Conv 5x5 -> 16 feature maps, then activation
    out_c3 = conv_forward(out_s2, W2, b2)        # shape (16, 10, 10)
    out_c3 = tanh(out_c3)
    
    # Layer S4: 2x2 Pooling -> 16 feature maps
    out_s4 = avg_pool_forward(out_c3)            # shape (16, 5, 5)
    
    # Layer C5: Conv 5x5 -> 120 feature maps (1x1 each), then activation
    out_c5 = conv_forward(out_s4, W3, b3)        # shape (120, 1, 1)
    out_c5 = tanh(out_c5)
    # Flatten output of C5 to a vector of length 120:
    out_c5_flat = out_c5.reshape(-1)             # shape (120,)
    
    # Layer F6: Fully connected -> 84, then activation
    out_f6 = fc_forward(out_c5_flat, W4, b4)     # shape (84,)
    out_f6 = tanh(out_f6)
    
    # Output layer: Fully connected -> 10, then softmax
    out_out = fc_forward(out_f6, W5, b5)         # shape (10,)
    probs = softmax(out_out)                     # shape (10,) probabilities
    
    return probs
